In [27]:
!pip install -U transformers
!pip install datasets
!pip install -U accelerate

In [1]:
import numpy as np
import pandas as pd
import tempfile
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer

2023-10-27 03:07:38.658813: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-27 03:07:38.698508: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-27 03:07:38.698540: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-27 03:07:38.698560: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-27 03:07:38.705199: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: A

In [29]:
pip install deepspeed==0.7.0

Note: you may need to restart the kernel to use updated packages.


In [30]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("amaydle/npc-dialogue")
train = dataset['train']
test = dataset['test']

In [58]:
testing_data

Dataset({
    features: ['Name', 'Biography', 'Query', 'Response', 'Emotion'],
    num_rows: 192
})

In [31]:
# Load the DailyDialog dataset
# dataset = load_dataset("amaydle/npc-dialogue")

# def concatenate_utterances(example):
#   example['full_dialog'] = " ".join(example['Query'] + example['Response'])
#   return example

# dataset = dataset.map(concatenate_utterances)
dataset = load_dataset("amaydle/npc-dialogue")
def concatenate_utterances_train(example):

  character_name = example['Name']
  biography = example['Biography']
  
  input_text = f"[Character: {character_name}][Biography: {biography}]" 
  #input_text += " ".join(example['Query'] + example['Response'])
  input_text += example['Query'] + example['Response'] 

  example['full_dialog'] = input_text


  return example

train = train.map(concatenate_utterances_train)


Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

In [33]:
# Load the DailyDialog dataset
# dataset = load_dataset("amaydle/npc-dialogue")

# def concatenate_utterances(example):
#   example['full_dialog'] = " ".join(example['Query'] + example['Response'])
#   return example

# dataset = dataset.map(concatenate_utterances)
def concatenate_utterances_test(example):

  character_name = example['Name']
  biography = example['Biography']
  
  input_text = f"[Character: {character_name}][Biography: {biography}]" 
  #input_text += " ".join(example['Query'] + example['Response'])
  input_text += example['Query']

  example['full_dialog'] = input_text


  return example

test = test.map(concatenate_utterances_test)


Map:   0%|          | 0/192 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'full_dialog'],
        num_rows: 1723
    })
    test: Dataset({
        features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'full_dialog'],
        num_rows: 192
    })
})

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'full_dialog'],
        num_rows: 1723
    })
    test: Dataset({
        features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'full_dialog'],
        num_rows: 192
    })
})

In [34]:
# Load the tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-small')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('microsoft/DialoGPT-small')

In [38]:
# Encode the full dialog
def encode(examples):
  encoded = tokenizer(examples['full_dialog'],
                      truncation=True,
                      padding='max_length',
                      max_length=500)

  encoded['labels'] = encoded['input_ids'][:]
  return encoded

# Apply functions
# dataset = dataset.map(concatenate_utterances)
encoded_dataset_train = train.map(encode, batched=True)
encoded_dataset_test = test.map(encode, batched=True)

Map:   0%|          | 0/1723 [00:00<?, ? examples/s]

Map:   0%|          | 0/192 [00:00<?, ? examples/s]

In [39]:
encoded_dataset_train

Dataset({
    features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'full_dialog', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1723
})

In [40]:
encoded_dataset_test

Dataset({
    features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'full_dialog', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 192
})

In [41]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'full_dialog', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1723
    })
    test: DatasetDict({
        train: Dataset({
            features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'input_ids', 'attention_mask', 'labels'],
            num_rows: 1723
        })
        test: Dataset({
            features: ['Name', 'Biography', 'Query', 'Response', 'Emotion', 'input_ids', 'attention_mask', 'labels'],
            num_rows: 192
        })
    })
})

In [34]:
import torch
print(torch.cuda.is_available())

True


In [42]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='output',
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    warmup_steps=100,
    weight_decay=0.01,
    gradient_accumulation_steps = 8,
    fp16 = True,
    #deepspeed = True
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test # use 'test' for evaluation
)

In [43]:
# Evaluate before fine-tuning
pre_eval_results = trainer.evaluate(encoded_dataset_test)

# Get predictions for test set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset_test.select(range(10)))

In [44]:
# Fine-tune the model
trainer.train()

Step,Training Loss
500,0.531900


TrainOutput(global_step=535, training_loss=0.505829283455822, metrics={'train_runtime': 331.444, 'train_samples_per_second': 25.992, 'train_steps_per_second': 1.614, 'total_flos': 2183217408000000.0, 'train_loss': 0.505829283455822, 'epoch': 4.97})

In [45]:
# Get predictions for validation set before fine tuning for 10 samples
pre_val_predictions = trainer.predict(encoded_dataset['test']['test'].select(range(10)))
# Evaluate after fine-tuning
post_eval_results = trainer.evaluate(encoded_dataset['test']['test'])

# Print the evaluation losses before and after fine-tuning
print('Evaluation Results before fine-tuning :', pre_eval_results['eval_loss'])
print('Evaluation Results after fine-tuning  :', post_eval_results['eval_loss'])

# Get predictions for validation set before fine tuning for 10 samples
post_val_predictions = trainer.predict(encoded_dataset['test']['test'].select(range(10)))

# Zip the pre and post tuning predictions
predictions = zip(pre_val_predictions.predictions, post_val_predictions.predictions)

Evaluation Results before fine-tuning : 5.609518527984619
Evaluation Results after fine-tuning  : 0.12609337270259857


In [46]:
for idx, (pre, post) in enumerate(predictions):
    pre_pred = tokenizer.decode(np.argmax(pre, axis=-1), skip_special_tokens=True)
    post_pred = tokenizer.decode(np.argmax(post, axis=-1), skip_special_tokens=True)
    # ground_truth = encoded_dataset['test']['test'][idx]
    
    # print('Ground truth \n' + ground_truth + '\n')
    print('Pre-prediction \n' + pre_pred + '\n')
    print('Post-prediction \n'+ post_pred + '\n')
    print('----------------------------------------------------------------------------------------------------------------------\n')

Pre-prediction 
Character: Araina Mathur][Biography: Naina Mathur is a determined and passionate teacher who has a stutter.]What is your most challenge you have in a teacher?Iquuring that student a the necessary's and seek to succeed.

Post-prediction 
Character: Araina Mathur][Biography: Naina Mathur is a determined and passionate teacher who has a stutter.]What is your most challenge you have in a teacher?Iquuring that student a the necessary's and seek to succeed.

----------------------------------------------------------------------------------------------------------------------

Pre-prediction 
Character: Ararayr][Biography: Zephyr is a mischievous fairy who loves playing pranks on unsuspecting travelers. Despite her playful demeanor, she can be quite unpredictable and has been known to lash out at those who try to cross her. Despite comes from a long line of fairies, and her ancestors were known for their powerful magic.]What isates you to keep pranks on unsuspecting?I's a a I 

In [17]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 26.7 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24932 sha256=dd709e9d8e8bfaeaa65d4820f118a49392b617b4d10a63c6f485fa8301a28c3c
  Stored in directory: /home/ubuntu/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [47]:
# After getting pre_val_predictions and post_val_predictions

# import evaluate
from datasets import load_metric

rouge = load_metric('rouge')

pre_results = rouge.compute(predictions=pre_val_predictions.predictions, 
                            references=post_val_predictions.predictions)

post_results = rouge.compute(predictions=post_val_predictions.predictions,
                             references=pre_val_predictions.predictions)

print("ROUGE scores before fine-tuning:")
print(pre_results)

print("ROUGE scores after fine-tuning:") 
print(post_results)

ROUGE scores before fine-tuning:
{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rouge2': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rougeL': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rougeLsum': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0))}
ROUGE scores after fine-tuning:
{'rouge1': AggregateScore(low=Score(precision=1.0, recall=1.0, fmeasure=1.0), mid=Score(precision=1.0, recall=1.0, fmeasure=1.0), high=Score(precision=1.0, recall=1.0, fmeasure=1.0)), 'rouge2': Aggre